## Install Dependencies

In [1]:
# Install dependencies
!pip install -e .

Obtaining file:///home/ec2-user/SageMaker/nano-graphrag
  Preparing metadata (setup.py) ... done
  Installing build dependencies ...done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached s3transfer-0.11.2-py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 150.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 163.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 145.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 139.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 131.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 144.2 MB/s eta 0:00:00
Using cached s3transfer-0.11.2-py3-none-any.whl (84 kB)
   ━

## Import Required Libraries

In [5]:
import os
import json
import time
import boto3
import logging
import litellm
from pathlib import Path
from litellm import completion, RateLimitError
from nano_graphrag import GraphRAG, QueryParam

## Setup Logging

In [6]:
logging.basicConfig(
    format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', 
    level=logging.INFO
)
logger = logging.getLogger(__name__)

### Define Environment Variable

In [7]:
os.environ["AWS_REGION_NAME"] = "us-east-1"

## Assume Cross-Account Role

In [8]:
# Read the IAM role from the BedrockCrossAccount.txt file
ROLE_TO_ASSUME = Path(os.path.join(os.environ["HOME"], "BedrockCrossAccount.txt")).read_text().strip()
logger.info(f"ROLE_TO_ASSUME={ROLE_TO_ASSUME}")

# Assume the IAM role using AWS STS
credentials = boto3.client('sts').assume_role(
    RoleArn=ROLE_TO_ASSUME,
    RoleSessionName='BedrockSession'
)['Credentials']

# Set environment variables for AWS credentials
os.environ["AWS_ACCESS_KEY_ID"] = credentials['AccessKeyId']
os.environ["AWS_SECRET_ACCESS_KEY"] = credentials['SecretAccessKey']
os.environ["AWS_SESSION_TOKEN"] = credentials['SessionToken']

INFO:__main__:ROLE_TO_ASSUME=arn:aws:iam::605134468121:role/BedrockCrossAccount
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [9]:
!aws sts get-caller-identity

{
    "UserId": "AROAYZZGTGQMSXMXAZQY7:BedrockSession",
    "Account": "605134468121",
    "Arn": "arn:aws:sts::605134468121:assumed-role/BedrockCrossAccount/BedrockSession"
}


## A wrapper function to invoke LiteLLM

In [10]:
logger.info(f"Role ARN read from file: {ROLE_TO_ASSUME}")

INFO:__main__:Role ARN read from file: arn:aws:iam::605134468121:role/BedrockCrossAccount


In [11]:
litellm.set_verbose=False

In [12]:
import litellm
from typing import Tuple
def get_inference(model_name: str, prompt: str, max_tokens: int = 100, temperature: float = 0.7, n: int = 1, top_p: float = 1, stream: bool = False) -> Tuple[litellm.types.utils.ModelResponse, float]:
    import sys
    import time
    max_retries = 10
    retry_delay = 60  # seconds
    
    for attempt in range(max_retries):
        try:
            # Your task processing code here
            print(f"model_name={model_name}")
            st = time.perf_counter()
            response = completion(
                    model=model_name,
                    model_id=None,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=max_tokens,
                    temperature=temperature,
                    n=n,
                    top_p=top_p,
                    stream=stream
                )
            latency = time.perf_counter() - st
            print(f"got response in {latency:0.2f}s")
            return response, latency
        except RateLimitError as e:
            if attempt < max_retries - 1:
                this_retry_delay = retry_delay * (attempt + 1) + random.randint(1, 10)
                print(f"rate limit hit for model_id {model_id} on attempt {attempt + 1}. Waiting {this_retry_delay} seconds...", file=sys.stderr, flush=True)
                time.sleep(this_retry_delay)
                continue
            else:
                print(f"max retries exceeded for model_id {model_id}", file=sys.stderr, flush=True)
                raise  # Re-raise the exception if we've exhausted all retries
                
        except Exception as e:
            print(f"Unexpected error processing model_id {model_id}: {e}", file=sys.stderr, flush=True)
            raise

## Read Data

In [13]:
!curl https://raw.githubusercontent.com/gusye1234/nano-graphrag/main/tests/mock_data.txt > book.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  184k  100  184k    0     0  1848k      0 --:--:-- --:--:-- --:--:-- 1866k


In [14]:
model_id_list = ["bedrock/us.meta.llama3-1-70b-instruct-v1:0",
                 "bedrock/us.meta.llama3-2-1b-instruct-v1:0",
                 "bedrock/us.meta.llama3-2-3b-instruct-v1:0",
                 "bedrock/us.meta.llama3-2-11b-instruct-v1:0",
                 "bedrock/us.meta.llama3-2-90b-instruct-v1:0",
                 "bedrock/us.anthropic.claude-3-5-sonnet-20241022-v2:0",
                 "bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0",
                 "bedrock/amazon.nova-lite-v1:0",
                 "bedrock/amazon.nova-micro-v1:0",
                 "bedrock/amazon.nova-pro-v1:0"]

## Run the Pipeline

In [15]:
!aws sts get-caller-identity

{
    "UserId": "AROAYZZGTGQMSXMXAZQY7:BedrockSession",
    "Account": "605134468121",
    "Arn": "arn:aws:sts::605134468121:assumed-role/BedrockCrossAccount/BedrockSession"
}


In [16]:
# import boto3
# from nano_graphrag import GraphRAG

# # Now, initialize GraphRAG
# graph_func = GraphRAG(
#     working_dir="./nano_graphrag_cache",
#     using_amazon_bedrock=True,
#     best_model_id="us.anthropic.claude-3-sonnet-20240229-v1:0",
#     cheap_model_id="us.anthropic.claude-3-haiku-20240307-v1:0"
# )

In [31]:
prompt = "the little red riding hood"
model_id = "us.meta.llama3-1-8b-instruct-v1:0"
max_tokens = 100
temperature = 0.7
n = 1
top_p = 1
stream = False
response, latency = get_inference(model_id, prompt, max_tokens, temperature, n, top_p, stream)

04:44:05 - LiteLLM:INFO: utils.py:2944 - 
LiteLLM completion() model= us.meta.llama3-1-8b-instruct-v1:0; provider = bedrock
INFO:LiteLLM:
LiteLLM completion() model= us.meta.llama3-1-8b-instruct-v1:0; provider = bedrock
INFO:botocore.credentials:Found credentials in environment variables.


model_name=us.meta.llama3-1-8b-instruct-v1:0


INFO:httpx:HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/us.meta.llama3-1-8b-instruct-v1:0/converse "HTTP/1.1 200 OK"
04:44:06 - LiteLLM:INFO: utils.py:1120 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


got response in 1.52s


In [32]:
def custom_bedrock_inference(model_id, prompt, max_tokens=100, temperature=0.7, n=1, top_p=1, stream=False):
    """
    Custom function to replace NanoGraphRAG's internal Bedrock calls.
    """
    from litellm import get_inference  # ✅ Import the working LiteLLM function

    # 🔥 Use the same model settings that worked previously
    response, latency = get_inference(
        model_id=model_id,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        n=n,
        top_p=top_p,
        stream=stream
    )

    logging.info(f"✅ Bedrock response received in {latency}s")
    return response

# ✅ Inject custom inference function into GraphRAG
graph_func.use_llm_func = custom_bedrock_inference
logging.info("✅ Successfully overridden NanoGraphRAG's Bedrock call!")

INFO:root:✅ Successfully overridden NanoGraphRAG's Bedrock call!


In [ ]:
# ✅ Load Sample Data
book_path = "book.txt"
if not Path(book_path).exists():
    raise FileNotFoundError(f"❌ File {book_path} not found!")

with open(book_path, "r", encoding="utf-8") as f:
    book_text = f.read()

# ✅ Try inserting text into GraphRAG
try:
    logging.info("📢 Inserting book text into GraphRAG...")
    graph_func.insert(book_text)  # 🚀 Now uses your working Bedrock setup!
    logging.info("✅ Data insertion completed successfully!")
except Exception as e:
    logging.error(f"❌ Failed to insert book text: {e}")
    raise

INFO:root:📢 Inserting book text into GraphRAG...
INFO:nano-graphrag:[New Docs] inserting 1 docs
INFO:nano-graphrag:[New Chunks] inserting 42 chunks
INFO:nano-graphrag:[Entity Extraction]...
